**DS 301: Applied Data Modeling and Predictive Analysis**

**Lecture 6 – Multiclass Classification**

# Multiclass Classifier with MNIST Dataset

Nok Wongpiromsarn, 2 September 2020

**Credit:** The large portion of the code has been taken from Chapter 3 of Aurélien Géron, Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow

**Load and process the mnist handwritten digit dataset**

In [ ]:
import numpy as np
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1)
x, y = mnist["data"], mnist["target"]
y = y.astype(np.uint8)
x_train, x_test, y_train, y_test = x[:60000], x[60000:], y[:60000], y[60000:]

**Train a Support Vector Machine classifier**

SVM is strictly a binary classifier. When using it for a multiclass classification, scikit-learn uses the OvO strategy: it trained 45 binary classifiers, got their decision scores for the image, and selected the class that won the most duels

In [ ]:
from sklearn.svm import SVC
svm_clf = SVC()
svm_clf.fit(x_train, y_train) # y_train, not y_train_5

In [ ]:
from common_plots import DigitPlotter

some_digit = x[0]
some_digit_label = y[0]
plotter = DigitPlotter((28, 28))
plotter.plot_single(some_digit, some_digit_label)

some_digit_scores_svm = svm_clf.decision_function([some_digit])
print("some_digit_score:", some_digit_scores_svm)
print("class with highest score: ", svm_clf.classes_[np.argmax(some_digit_scores_svm)])

**Train an SGD classifier**

In [ ]:
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier(max_iter=1000, tol=1e-3, random_state=42)
sgd_clf.fit(x_train, y_train)

In [ ]:
some_digit_scores_sgd = sgd_clf.decision_function([some_digit])
print("some_digit_score:", some_digit_scores_sgd)
print("class with highest score: ", sgd_clf.classes_[np.argmax(some_digit_scores_sgd)])

**Compute the confusion matrix**

In [ ]:
from sklearn.model_selection import cross_val_predict
y_train_pred = cross_val_predict(sgd_clf, x_train, y_train, cv=3)

from sklearn.metrics import confusion_matrix
conf_mx = confusion_matrix(y_train, y_train_pred)
print(conf_mx)

**Visualize the confusion matrix**

In [ ]:
import matplotlib.pyplot as plt
plt.matshow(conf_mx, cmap=plt.cm.gray)
plt.show()

**Focus on the error rate**

In [ ]:
# Compute the error rate on each row
row_sums = conf_mx.sum(axis=1, keepdims=True)
norm_conf_mx = conf_mx / row_sums

# Fill the diagonal with zeros to keep only the errors, and plot the result
np.fill_diagonal(norm_conf_mx, 0)
plt.matshow(norm_conf_mx, cmap=plt.cm.gray)
plt.show()